# <center> Taller Keras $-$ Ejemplo 1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from time import time

from tensorflow.keras.models import Model, Sequential, load_model, save_model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical

from pandas import DataFrame
from sklearn.metrics import confusion_matrix, classification_report
from plotmatrix import pretty_plot_confusion_matrix, PlotMatrix

import warnings
warnings.filterwarnings('ignore')

# Introducción a las Redes Neuronales (NNs)

__Objetivos:__
* Implementar una NN de varias capas en Keras para clasificar la base de datos MNIST.

In [ ]:
# cargar la interfaz a la base de datos que viene con Keras
from tensorflow.keras.datasets import mnist

# lectura de los datos
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
print('Raw data shapes:')
print('X train:', train_images.shape, 'Y train:', train_labels.shape)
print('X test:', test_images.shape, 'Y test:', test_labels.shape)

# pre-procesamiento de los datos
train_images = train_images.reshape((60000, -1))
train_images = train_images.astype('float32') / 255.

test_images = test_images.reshape((10000, -1))
test_images = test_images.astype('float32') / 255.

# one-hot encoding
from tensorflow.keras.utils import to_categorical
train_labels = to_categorical(train_labels)
test_labels  = to_categorical(test_labels)

In [ ]:
print('Modified data shapes:')
print('X train:', train_images.shape, 'Y train:', train_labels.shape)
print('X test:', test_images.shape, 'Y test:', test_labels.shape)

data_shape = train_images.shape[1:]

print('')
print('data shape:', data_shape)

In [ ]:
test_labels[0]

## Red neuronal multicapa

![](nn2.png)

__Ejercicio:__ Diseñe una red neuronal con 4 capas ocultas, más una capa de salida (como en la figura). Las capas deben ser densas. El número de neuronas en las capas debe ser: 512, 512, 256, 128, y 10 en la capa de salida.

Use funciones de activación ReLU (excepto en la capa de salida, donde se usará softmax). 


In [ ]:
#Ejemplo de cómo definir una arquitectura sin usar "Sequential"

def NNmulticapa(input_shape):
    I = Input(shape=input_shape, name='input')
    X = Dense(16, activation='relu', name='dense1')(I)
    X = Dense(10, activation='relu', name='classifier')(X)
    model = Model(I, X, name='NN-multicapa')
    return model

In [ ]:
data_shape

In [ ]:
if 'network2' in globals(): del network2
network2 = None

network2 = NNmulticapa(data_shape)

In [ ]:
network2.summary()

__Ejercicio:__ Compile y entrene la red neuronal de 2 capas.

Utilice el optimizador 'RMSprop', usted elige el learning rate apropiado. Use la función de pérdida 'categorical_crossentropy', y use como métrica el 'accuracy'.

Para el entrenamiento, use 15 épocas (iteraciones), un tamaño de batch de 64 ó 128, y haga una partición de validación de 0.16666 (5/6).

### Compile

In [ ]:
alpha = 1e-4
opt = optimizers.RMSprop(lr=alpha)

network2.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

### Train

In [ ]:
tic = time()
history2 = network2.fit(x=train_images, y=train_labels, epochs=20, batch_size=128, validation_split=0.1666)
toc = time()
print('total training time:', toc-tic, 'seconds')

### Ver el historial de desempeño de la red

In [ ]:
history_dict = history2.history
history_dict.keys()

In [ ]:
acc = history2.history['accuracy']
val_acc = history2.history['val_accuracy']
loss = history2.history['loss']
val_loss = history2.history['val_loss']

epochs = range(1, len(acc)+1)

# figure
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.subplot(1,2,2)
plt.plot(epochs, acc, 'ro', label='Training accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

### Resultados

In [ ]:
#results2 = network2.evaluate(test_images, test_labels)
#print('results loss:', results2[0])
#print('results accuracy:', results2[1])

In [ ]:
# compute predictions (inference process)
predictions2 = network2.predict(test_images)

# from predictions compute most probable class
pred2 = np.argmax(predictions2, 1)
test_labs = np.argmax(test_labels, 1)

In [ ]:
print(classification_report(test_labs, pred2))

In [ ]:
CM2 = confusion_matrix(test_labs, pred2)

# plot confusion matrix
cf2 = DataFrame(CM2)
pretty_plot_confusion_matrix(cf2, annot=True, pred_val_axis='x', figsize=(10,10), fz=12)

In [ ]:
# second plot
PlotMatrix(CM2, figsize=(10,10), cmap=plt.cm.Blues, title='Confusion Matrix', fz=15)

---

---

### Ejercicio 1

* Implementar una arquitectura de red multicapa con 2 capas intermedias (más la capa clasificadora).

* Usar las funciones de activación que usted desee. En la capa clasificadora, usar 'softmax'.

* Elegir los parámetros a su discreción: optimizador, tamaño de paso (*learning rate*), número de épocas, ...

* Mostrar los resultados del desempeño en entrenamiento y prueba. El objetivo es diseñar una red que tenga un mejor desempeño que la de la clase pasasa.

### Ejercicio 2

* Con la red del ejercicio anterior, experimentar usando diferentes clasificadores, para encontrar aquel que mejora el desempeño de su red.

* Experimente cambiando también otros parámetros: alpha, número de epochs, batch_size.

### Ejercicio 3

* Implementar una arquitectura de red multicapa con 4 capas intermedias (más la capa clasificadora).

* Usar las funciones de activación que usted desee. En la capa clasificadora, usar 'softmax'.

* Elegir los parámetros a su discreción: optimizador, tamaño de paso (*learning rate*), número de épocas, ...

* Mostrar los resultados del desempeño en entrenamiento y prueba. El objetivo es diseñar una red que tenga un mejor desempeño que el Ejercicio 1.